In [ ]:
from sklearn.manifold import TSNE
import numpy as np

X = np.random.random((100, 20))

n_components = 2  
perplexity = 30 

tsne = TSNE(n_components=n_components, perplexity=perplexity)
X_embedded = tsne.fit_transform(X)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], marker='.')
plt.title('t-SNE Visualization')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()